In [1]:
from openai import OpenAI
import pandas as pd
import json
import ast
from dotenv import load_dotenv
from tqdm import tqdm
import os

load_dotenv()

my_key = os.getenv("MY_API_KEY")

In [2]:
client = OpenAI(api_key=my_key)

In [ ]:
batch_size = 100   # how many rows per API call
labels = []

In [4]:
df = pd.read_csv("../Data/combined_twitch_data.csv")

In [12]:
MODEL = "gpt-4o-mini"
df = pd.read_csv("../Data/combined_twitch_data.csv")

for i in tqdm(range(0, len(df), batch_size), desc="Batch labeling"):
    batch_texts = df["message"].iloc[i:i+batch_size].tolist()

    # Construct the batch prompt
    prompt = "Classify each of the following chat messages as Offensive (1) or Not Offensive (0).\n"
    prompt += "Return only a Python-style list of 0s and 1s, in the same order. No need to include Markdown style in answer (e.g. ```python ....```)\n\n"

    for j, text in enumerate(batch_texts, start=1):
        prompt += f"{j}. {text}\n"

    # Call OpenAI
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a moderation classifier."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
    )

    # Parse the output
    output = response.choices[0].message.content.strip()
    try:
        # batch_labels = eval(output)  # risky but works if model outputs [0,1,0,...]
        batch_labels = ast.literal_eval(output)  # safer
    except:
        # fallback if parsing fails
        batch_labels = [int(x) for x in output.replace("[","").replace("]","").split(",")]

    labels.extend(batch_labels)

Batch labeling:  61%|██████    | 83/136 [27:11<17:21, 19.65s/it]


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-PQ98pCLKujxxBRqnNgeNsh5z on tokens per min (TPM): Limit 100000, Used 99722, Requested 373. Please try again in 41m2.4s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [8]:
output_strip = output.split('\n')

In [15]:
len(labels)

4985

In [16]:
# reduce df to the size of labels in case of any mismatch
df = df.iloc[:len(labels)]


In [19]:
df["label"] = labels

In [20]:
output_name = "../Data/combined_twitch_data_labeled_by_GPT.csv"
df.to_csv(output_name, index=False)

In [22]:
df['label'].value_counts()

label
0    4658
1     327
Name: count, dtype: int64